# 9. 텐서플로 시작하기

## 9.2 첫 번째 계산 그래프를 만들어 세션에서 실행하기

In [1]:
import tensorflow as tf

In [2]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x * x * y + y + 2

- 실제로는 어떤 계산도 수행하지 않는다. 단지 계산 그래프만 만들 뿐이다. 사실 변수조차도 초기화되지 않는다. 이 계산 그래프를 평가하려면 텐서플로 **세션**을 시작하고 변수를 초기화한 다음 f를 **평가**(evaluation)해야 한다.

- 텐서플로 세션은 연산을 CPU나 GPU같은 **장치**에 올리고 실행하는 것을 도와주며 모든 변숫값을 가지고 있다. (분산 환경에서는 서버)

세션 만들고 변수를 초기화한 다음 f를 평가하고 세션을 닫기(자원 해제)

In [3]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print (result)
sess.close()

42


- 매번 sess.run() 을 번복하기 번거롭다. with를 사용하자

In [4]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

- 각 변수의 초기화를 일일이 실행하는 대신 global_variables_initializer() 함수를 사용할 수 있다. 초기화를 바로 수행하지 않고 계산 그래프가 실행될 때 모든 변수를 초기화할 노드를 생성한다.

In [5]:
init = tf.global_variables_initializer() # init 노드 준비

with tf.Session() as sess:
    init.run()
    result = f.eval()

- 보통 텐서플로 프로그램은 계산 그래프를 만드는 **구성 단계**, 이 그래프를 실행하는 **실행 단계**로 나뉜다.
- 구성 단계에서는 훈련에 필요한 계산과 머신러닝 모델을 표현한 계산 그래프를 만든다.
- 실행 단계에서는 훈련 스텝을 반복해서 평가하고 모델 파라미터를 점진적으로 개선하기 위해 반복 루프를 수행한다.

## 9.3 계산 그래프 관리

노드를 만들면 자동으로 기본 계산 그래프에 추가된다.

In [7]:
x1 = tf.Variable(1)

In [9]:
x1.graph is tf.get_default_graph()

True

독립적인 계산 그래프를 여러개 만들어야 할 때는 새로운 Graph 객체를 만들어 with 블록 안에서 임시로 이를 기본 계산 그래프로 사용할 수 있다.

In [10]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [11]:
x2.graph is graph

True

In [14]:
x2.graph is tf.get_default_graph()

False

## 9.4 노드 값의 생애주기

한 노드를 평가할 때 텐서플로는 이 노드가 의존하고 있는 다른 노드들을 자동으로 찾아 먼저 평가한다.

In [16]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print (y.eval()) # 10
    print (z.eval()) # 15

10
15


- 먼저 간단한 그래프를 정의한다.
- 이후 세션을 시작하고 y를 평가하기 위해 계산 그래프를 실행한다.
- 텐서플로우는 y가 x에 의존한다는 것과, x는 w에 의존한다는 것을 감지한다.
- w를 평가하고, 그 다음에 x를, 그 다음에 y를 평가해서 y 값을 반환한다.
- z를 평가하기 위해 그래프를 실행한다.
- 텐서플로우는 w와 x를 먼저 평가해야 한다는 것을 감지한다.
- ** 이전에 평가된 w와 x를 재사용 하지 않는다.** (w와 x를 두번 평가한다.)
- 모든 노드의 값을 계산 그래프 실행 간에 유지되지 않는다. 다만 변숫값은 예외이며 그래프 실행간에도 세션에 의해 유지된다.
- (변수는 초기화 될때 일생이 시작되고 세션이 종료도리 때까지 남아 있다.)

w와 x를 두번 평가하지 않고 y와 z를 효율적으로 평가하려면 텐서플로가 한 번의 그래프 실행에서 y와 z를 모두 평가하도록 만들어야 한다.

In [17]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print (y_val)
    print (z_val)

10
15


## 9.5 텐서플로를 이용한 선형 회귀

- 상수와 변수 연산은 입력이 없다.(**소스 연산** 이라고 한다.)
- 입력과 출력은 **텐서**라는 다차원 배열이다.

- 데이터셋을 추출하고 모든 훈련 샘플에 편향에 대한 입력($x_0$ = 1)을 추가한다. (이 부분은 넘파이를 사용하므로 즉시 실행됨)
- 텐서플로 상수 노드 x와 y를 만들고 데이터와 타깃을 담는다.
- 텐서플로에서 행렬 연산을 사용해 theta를 정의한다.
- 행렬 함수는 transpose(), matmul(), matrix_inverse()인데 앞서 언급한 것처럼 계산을 즉각 수행하지는 않는다. 다만 그래프가 실행될 때 계싼을 수행할 노드를 생성한다.
- 마지막에 코드는 세션을 만들고 theta를 평가한다.

In [43]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

- GPU가 있는 경우 텐서플로가 자동으로 이 코드를 GPU에서 실행한다.

## 9.6 경사 하강법 구현

정규방정식 대신 경사하강법을 사용해 보자. 먼저 그래디언트를 수동으로 계산해 보고 그 다음에 텐서플로의 자동 미분 기능을 사용해 그래디언트를 자동으로 계산해 보자. 마지막으로 텐서플로에 내장된 옵티마이저(optimizer)를 사용하자.

```
CAUTION 경사 하강법을 사용할 때는 입력 벡터를 정규화하는 것이 중요하다. 그렇지 않으면 훈련 속도가 매우 느려진다. 정규화는 텐서플로나 넘파이, 사이킷런의 StandardScaler 또는 선호하는 다른 도구를 사용해도 가능하다.
```

In [45]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

### 9.6.1 직접 그래디언트 계산

- random_uniform() 함수는 난수를 담은 텐서를 생성하는 노드를 그래프에 생성한다. 넘파이의 rand() 함수처럼 크기와 난수의 범위를 입력받는다.
- assign() 함수는 변수에 새로운 값을 할당하는 노드를 생성한다. 여기서는 배치 경사 하강법의 스텝을 구현한다.
- 반복 루프는 훈련 단계를 계속 반복해서 실행하고(n_epoch 만큼), 100번 반복마다 현재의 평균제곱 에러(코드에서 mse 변수)를 출력한다. MSE는 매 반복에서 값이 줄어들어야 한다.
- reduce_mean() 함수는 텐서의 평균을 계산하는 노드를 그래프에 추가한다.
- 만약 assign() 함수를 사용하지 않고 theta = theta - learning_rate * gradients 라고 쓰면 theta는 더 이상 변수가 아니라 tf.subtact() 연산의 출력을 가리키는 텐서(tf.Tensor)가 된다.
- 텐서플로의 변수와 텐서 객체를 계산 그래프와 연산 노드를 가리키는 일종의 핸들이라고 생각하자.

In [46]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+ 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print ("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

Epoch 0 MSE = 8.667914
Epoch 100 MSE = 0.7233769
Epoch 200 MSE = 0.58784777
Epoch 300 MSE = 0.5678451
Epoch 400 MSE = 0.55565375
Epoch 500 MSE = 0.5469541
Epoch 600 MSE = 0.5406787
Epoch 700 MSE = 0.5361475
Epoch 800 MSE = 0.5328738
Epoch 900 MSE = 0.5305092


In [44]:
theta_value

array([[-3.7185181e+01],
       [ 4.3633747e-01],
       [ 9.3952334e-03],
       [-1.0711310e-01],
       [ 6.4479220e-01],
       [-4.0338000e-06],
       [-3.7813708e-03],
       [-4.2348403e-01],
       [-4.3721911e-01]], dtype=float32)

### 9.6.2 자동 미분 사용

텐서프롤의 자동 미분 기능

In [47]:
gradients = tf.gradients(mse, [theta])[0]

- gradients() 함수는 하나의 연산(여기서는 mse)과 변수 리스트(여기서는 theta 하나)를 받아 각 변수에 대한 연산의 그래디언트를 계산하는 새로운 연산을 만든다. (변수당 하나씩)

- gradients 노드는 theta에 대한 MSE의 그래디언트 벡터를 계산한다.

- **후진 자동 미분**(reverse-mode autodiff)를 통해 모든 입력에 대한 출력의 편미분을 $n_{output}$ + 1 그래프 순회안에 모두 계산한다.

### 9.6.3 옵티마이저 사용

텐서플로는 자동으로 그래디언트를 계산해 주지만 더 쉬운 방법이 있다.

- 텐서플로는 경사 하강법 옵티마이저를 포함하여 여러 가지 내장 옵티마이저를 제공한다.

In [48]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

- 다른 옵티마이저를 사용하고 싶으면 한 줄만 바꾸면 된다.

In [49]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)

## 9.7 훈련 알고리즘에 데이터 주입

- 미니배치 경사 하강법을 적용하려면 매 반복에서 X와 y를 다음번 미니배치로 바꾸어 주어야 한다. 이를 위해 플레이스홀더 노드를 사용한다.
- 크기를 지정하여 강제할 수 있고, 차원을 None으로 설정하면 어떤 크기도 가능하다는 뜻이 된다.

- 아래 코드에서 A는 랭크가 2(즉, 2차원)이고 열은 3개여야 한다. 하지만 행의 개수는 상관없다.

In [52]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5

with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})

print (B_val_1)

print (B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


미니배치 경사 하강법을 위해서 기존 코드를 수정하자.

In [53]:
X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

배치 크기와 전체 배치 횟수를 정의한다

In [54]:
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [ ]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown, 20640을 최대로 100개 뽑아내기
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

## 9.8 모델 저장과 복원

- 모델을 훈련시키고 나면 필요할 때 다시 쓸 수 있도록 모델 파라미터를 디스크에 저장해야 한다. 또는 훈련하는 동안 일정한 간격으로 체크포인트를 저장해두면 컴퓨터가 훈련 중간에 문제를 일으켜도 처음부터 다시 시작하지 않고 마지막 체크포인트부터 이어나갈 수 있다.

- 텐서플로에서 모델을 저장하기 위해서는, 구성 단계의 끝에서 (모든 변수 노드를 생성한 후) Saver 노드를 추가하고, 실행 단계에서 모델을 저장하고 싶을 때 save() 메서드에 세션과 체크파인트 파일의 경로를 전달하여 호출하면 된다.

In [83]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.const